In [261]:
import pandas as pd
import numpy as np
import regex as re

In [262]:
# pandas.errors.UndefinedVariableError occurs when the variable or column name does not exist
df = pd.DataFrame({"RCFD1111": [1, 2, 3], "RCFD2222": [1, 2, 3], "RCFD3333": [1, 2, 3]})
try:
    eval_string = "RCFD1111 + RCFD5555"
    df[eval_string] = df.eval(eval_string)
except Exception as e:
    print(type(e))
df


<class 'pandas.errors.UndefinedVariableError'>


,RCFD1111,RCFD2222,RCFD3333
0,1,1,1
1,2,2,2
2,3,3,3


In [263]:
# Any value containg a null, when evaluated, will always result in a null
df = pd.DataFrame({"RCFD1111": [1, 2, 3], "RCFD2222": [np.nan, 2, 3], "RCFD3333": [1, 2, 3]})
try:
    eval_string = "RCFD1111 + RCFD2222"
    df[eval_string] = df.eval(eval_string)
except Exception as e:
    print(type(e))
df


,RCFD1111,RCFD2222,RCFD3333,RCFD1111 + RCFD2222
0,1,NaN,1,NaN
1,2,2.0,2,4.0
2,3,3.0,3,6.0


In [264]:
# Missing column name or variable will always have precedence over null values
df = pd.DataFrame({"RCFD1111": [1, 2, 3], "RCFD2222": [np.nan, 2, 3], "RCFD3333": [1, 2, 3]})
try:
    eval_string = "RCFD1111 + RCFD2222 + RCFD5555"
    df[eval_string] = df.eval(eval_string)
except Exception as e:
    print(type(e))
df


<class 'pandas.errors.UndefinedVariableError'>


,RCFD1111,RCFD2222,RCFD3333
0,1,NaN,1
1,2,2.0,2
2,3,3.0,3


In [265]:
# If the denominator contains a 0, the resulting value will be inf
df = pd.DataFrame({"RCFD1111": [1, 2, 3], "RCFD2222": [1, 2, 3], "RCFD3333": [0, 2, 3]})
try:
    eval_string = "RCFD1111 / RCFD3333"
    df[eval_string] = df.eval(eval_string)
except Exception as e:
    print(e)
    print(type(e))
df

,RCFD1111,RCFD2222,RCFD3333,RCFD1111 / RCFD3333
0,1,1,0,inf
1,2,2,2,1.0
2,3,3,3,1.0


In [266]:
# If the denominator contains a null, the resulting value will be null
df = pd.DataFrame({"RCFD1111": [1, 2, 3], "RCFD2222": [1, 2, 3], "RCFD3333": [np.nan, 2, 3]})
try:
    eval_string = "RCFD1111 / RCFD3333"
    df[eval_string] = df.eval(eval_string)
except Exception as e:
    print(e)
    print(type(e))
df

,RCFD1111,RCFD2222,RCFD3333,RCFD1111 / RCFD3333
0,1,1,NaN,NaN
1,2,2,2.0,1.0
2,3,3,3.0,1.0


In [267]:
# 1. Using regex, extract the column names for individual MDRM codes
pattern = r'^[A-Z]{4}[A-Z0-9]{4}'

# If the denominator contains a null, the resulting value will be null
df = pd.DataFrame({"RCFD1111": [1, 2, 0], "RCFD2222": [1, 2, 3], "RCFD3333": [np.nan, 0, 0]})
try:
    eval_string = "RCFD1111 / RCFD3333"
    mdrm_columns = re.findall(pattern, eval_string)
    df[eval_string] = df.eval(eval_string)
except Exception as e:
    print(e)
    print(type(e))
df

,RCFD1111,RCFD2222,RCFD3333,RCFD1111 / RCFD3333
0,1,1,NaN,NaN
1,2,2,0.0,inf
2,0,3,0.0,NaN


### Business Logic:
  * Suppose an expression for getting the sum of two columns is `RCFD1111 + RCFD2222` and the column `RCFD2222` does not exist but `RCFD1111` does. The result should not be null.
  * Suppose an expression for getting the sum of two columns is `RCFD1111 + RCFD2222` and both `RCFD1111` and `RCFD2222` do not exist. This should result in a null value.
  * Suppose an expression for getting the qoutient of two columns `RCFD1111 / RCFD2222` and either the numerator or denominator column does not exist. The resulting value should be null.
  * Suppose an expression for getting the qoutient of two columns `RCFD1111 / RCFD2222` and the numerator is zero and the denominator is non-null value. The resulting value should be zero.
  * Suppose an expression for getting the qoutient of two columns `RCFD1111 / RCFD2222` and the numerator is non-zero and the denominator is zero. The resulting value is infinite and should be converted to null.

### Solution

### Metadata Preparation
1. Create columns in the metadata for numerator and denominator
2. Break down the MDRM Code expression into Numerator and Denominator
3. Example 1: "a + b" will have numerator equal to "a + b" and denominator equal to "1"
4. Example 2: "(a + b)/c" will have a numerator equal to "a + b" and denominator equal to "c"


### Process
1. Evaluate the numerator expression available in the metadata. 
2. If all of the MDRM code in the numerator does not exist yet in Attic, the overall expression should return null.
3. If at least one MDRM code in the numerator exist, fill the missing MDRM code with zero<sup>1</sup>.
5. Evaluate the denominator expression available in the metadata.
4. If all of the MDRM code in the denominator does not exist yet in Attic, the overall expression should return null.
5. If at least one MDRM code in the denominator exist, fill the missing MDRM code with zero.
5. If the denominator result is 0, the overall expression should return null.

<sup>1</sup> When pandas.eval evaluates an expression that contains a null value, it will result in a null value. Hence, 0 is used as a replacement for null.




In [268]:
# Create a dataframe
df = pd.DataFrame({"RCFD1111": [1, np.nan, np.nan], "RCFD2222": [1, np.nan, 3], "RCFD3333": [0, 2, 3]})
df

,RCFD1111,RCFD2222,RCFD3333
0,1.0,1.0,0
1,NaN,NaN,2
2,NaN,3.0,3


In [269]:
df_mapping = pd.DataFrame(
    {
        "MDRM Code": ["RCFD1111 + RCFD2222", "(RCFD1111 + RCFD2222) / RCFD3333"],
        "Numerator": ["RCFD1111 + RCFD2222", "(RCFD1111 + RCFD2222)"],
        "Denominator": [1, "RCFD3333"],
    }
)
df_mapping

,MDRM Code,Numerator,Denominator
0,RCFD1111 + RCFD2222,RCFD1111 + RCFD2222,1
1,(RCFD1111 + RCFD2222) / RCFD3333,(RCFD1111 + RCFD2222),RCFD3333


In [270]:
for index, row in df_mapping.iterrows():
    mdrm_code_expression = row["MDRM Code"]
    numerator_expression = row["Numerator"]
    denominator_expression = row["Denominator"]

    # No need to evaluate already existing expression in the dataframe columns
    if mdrm_code_expression in df.columns:
        continue
    
    # These returns pandas series of equivalent to the result of an expression when evaluated.
    # Example: The expression RCFD1111 + RCFD2222 will return a series containing the sum of columns RCFD1111 and RCFD2222
    numerator_values = df.eval(numerator_expression)
    denominator_values = df.eval(denominator_expression)
    
    df[mdrm_code_expression] = numerator_values / denominator_values

In [271]:
df

,RCFD1111,RCFD2222,RCFD3333,RCFD1111 + RCFD2222,(RCFD1111 + RCFD2222) / RCFD3333
0,1.0,1.0,0,2.0,inf
1,NaN,NaN,2,NaN,NaN
2,NaN,3.0,3,NaN,NaN


The problem with this is that in the last row, the sum of `RCFD1111 + RCFD2222` is `NaN` instead of 3.

To solve this, we need to make sure that when there is at least 1 non-null value in a summation expression, it should not return in a null.

In [272]:
# Create a dataframe
df = pd.DataFrame({"RCFD1111": [1, np.nan, np.nan, 4], "RCFD2222": [1, np.nan, 3, 4], "RCFD3333": [0, 2, 3, np.nan]})
df

,RCFD1111,RCFD2222,RCFD3333
0,1.0,1.0,0.0
1,NaN,NaN,2.0
2,NaN,3.0,3.0
3,4.0,4.0,NaN


### Create a function for handling at least one existing MDRM Code logic

In [273]:
def evaluate_expression(df_input: pd.DataFrame, expression):
    """
    Evaluates the given expression on an input DataFrame, with additional handling for MDRM codes.

    Parameters:
        df_input (pd.DataFrame): The input DataFrame to evaluate the expression on.
        expression (str): The expression to evaluate.
    """

    df_input = df_input.copy()  # Needs to create a copy to avoid modifying the original referenced dataframe
    pattern = r'[A-Z]{4}[A-Z0-9]{4}'    # Regex pattern for an MDRM Code (i.e., RCFD1234)
    mdrm_columns = re.findall(pattern, str(expression))     # Get the list of MDRM codes present in the expression

    if mdrm_columns:
        mask = df_input[mdrm_columns].isna().all(axis=1)    # Mask rows where all of the indicated columns have null values
        df_input.loc[~mask, mdrm_columns] = df_input.loc[~mask, mdrm_columns].fillna(0)

    return df_input.eval(expression)


In [274]:
for index, row in df_mapping.iterrows():
    mdrm_code_expression = row["MDRM Code"]
    numerator_expression = row["Numerator"]
    denominator_expression = row["Denominator"]

    # No need to evaluate already existing expression in the dataframe columns
    if mdrm_code_expression in df.columns:
        continue
    
    # These returns pandas series of equivalent to the result of an expression when evaluated.
    # Example: The expression RCFD1111 + RCFD2222 will return a series containing the sum of columns RCFD1111 and RCFD2222
    numerator_values = evaluate_expression(df, numerator_expression)
    denominator_values = evaluate_expression(df, denominator_expression)  

    df[mdrm_code_expression] = numerator_values / denominator_values

In [275]:
df

,RCFD1111,RCFD2222,RCFD3333,RCFD1111 + RCFD2222,(RCFD1111 + RCFD2222) / RCFD3333
0,1.0,1.0,0.0,2.0,inf
1,NaN,NaN,2.0,NaN,NaN
2,NaN,3.0,3.0,3.0,1.0
3,4.0,4.0,NaN,8.0,NaN


### Do dataframe clean-up
* Convert inf to NaN

In [276]:
df = df.replace(np.inf, np.nan)
df

,RCFD1111,RCFD2222,RCFD3333,RCFD1111 + RCFD2222,(RCFD1111 + RCFD2222) / RCFD3333
0,1.0,1.0,0.0,2.0,NaN
1,NaN,NaN,2.0,NaN,NaN
2,NaN,3.0,3.0,3.0,1.0
3,4.0,4.0,NaN,8.0,NaN
